In [6]:
from dotenv import load_dotenv
load_dotenv()

True

# Replicate x Whisper

## Wishlist
- [ ] When querying these chunks consecutive chunks need to be grouped because the chunks are too small
			
	Idea 1:
	- Store these small chunks as is
	- Chunk the whole transcript, embed and store like a normal txt file
	- Extract chunks from the embedded ones and then get the start and end timestamps by comparing the text with the stored small chunks

In [7]:
import replicate
import json

In [ ]:
# output = replicate.run(
#     "openai/whisper:91ee9c0c3df30478510ff8c8a3a545add1ad0259ad3a9f78fba57fbc05ee64f7",
#     input={"audio": open("./data/youtube/Sal Khan's thoughts on mastery learning.m4a", "rb")}
# )
# print(output)
# with open("./data/youtube/Sal Khan's thoughts on mastery learning - Replicate x Whisper API Response.json", 'w') as f:
# 	json.dump(output, f)

In [31]:
# with open("./data/youtube/Sal Khan's thoughts on mastery learning - Transcription.txt", "w") as f:
# 	f.write(output['transcription'])

# with open("./data/youtube/Sal Khan's thoughts on mastery learning - Segments.txt", "a") as f:
# 	for chunk in output['segments']:
# 		f.write(chunk['text'] + "\n")

In [8]:
output = json.load(open("./data/youtube/Sal Khan's thoughts on mastery learning - Replicate x Whisper API Response.json", 'r'))

In [30]:
list(output.keys())

['detected_language', 'segments', 'transcription', 'translation']

In [5]:
output['segments'][0]

{'avg_logprob': -0.3576602739157136,
 'compression_ratio': 1.6016260162601625,
 'end': 7,
 'id': 0,
 'no_speech_prob': 0.00015473616076633334,
 'seek': 0,
 'start': 0,
 'temperature': 0,
 'text': ' This idea of mastery learning was always kind of this gold standard.',
 'tokens': [50364,
  639,
  1558,
  295,
  37951,
  2539,
  390,
  1009,
  733,
  295,
  341,
  3821,
  3832,
  13,
  50714]}

In [21]:
transcript = "\n".join([segment['text'] for segment in output['segments']])
transcript = transcript.strip()

In [34]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

In [14]:
textSplitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

In [22]:
transcriptChunks = textSplitter.create_documents([ transcript ])
transcriptChunks

[Document(page_content="This idea of mastery learning was always kind of this gold standard.\n This was actually as a part of a fellowship I had while I was at MIT called the El Orante Fellowship\n to make a learning software for students with ADHD.\n It is X plus 5 times X plus...\n And it immediately struck a chord with me because going into that,\n the whole premise of the software that I was working on was this idea that\n it's not that students aren't capable of learning some advanced mathematics", metadata={}),
 Document(page_content="it's not that students aren't capable of learning some advanced mathematics\n or that the topics are actually difficult,\n it's more that they just have gaps in their knowledge.\n And I did a lot of tutoring in high school and I saw that over and over again\n that the reason why students were having...\n My friends were having trouble with algebra or geometry.\n It was just because they had a gap in their negative numbers\n or dividing decimals or l

In [32]:
SEGMENT_OVERLAP_SECONDS = 2
chunksWithSegments = []

for chunk in transcriptChunks:
    chunkText = chunk.page_content

    chunkSegments = []
    for segment in output['segments']:
        if segment['text'].strip() in chunkText:
            chunkSegments.append(segment)

    # Store consecutive segments as one segment
    consecutiveSegments = [{
        'start': chunkSegments[0]['start'],
        'end': chunkSegments[0]['end']
    }]

    for segment in chunkSegments[1:]:
        lastSegmentEnd = consecutiveSegments[-1]['end']
        if segment['start'] in range(lastSegmentEnd, lastSegmentEnd + SEGMENT_OVERLAP_SECONDS):
            consecutiveSegments[-1]['end'] = segment['end']
        else:
            consecutiveSegments.append({
                'start': segment['start'],
                'end': segment['end']
            })
    
    chunksWithSegments.append(Document(
        page_content=chunkText,
        metadata={
        'segments': consecutiveSegments
        })
    )


In [33]:
for chunk in chunksWithSegments:
    print(chunk)

page_content="This idea of mastery learning was always kind of this gold standard.\n This was actually as a part of a fellowship I had while I was at MIT called the El Orante Fellowship\n to make a learning software for students with ADHD.\n It is X plus 5 times X plus...\n And it immediately struck a chord with me because going into that,\n the whole premise of the software that I was working on was this idea that\n it's not that students aren't capable of learning some advanced mathematics" metadata={'segments': [{'start': 0, 'end': 31}]}
page_content="it's not that students aren't capable of learning some advanced mathematics\n or that the topics are actually difficult,\n it's more that they just have gaps in their knowledge.\n And I did a lot of tutoring in high school and I saw that over and over again\n that the reason why students were having...\n My friends were having trouble with algebra or geometry.\n It was just because they had a gap in their negative numbers\n or dividing